In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# 广告点击情况
ctr_data = pd.read_csv('effect_tb.csv',header=None)
ctr_data.columns = ['dt','user_id','label','dmp_id']

# user_id:支付宝用户的id
# label表示用户在当天 dt 是否点击了活动广告。
# dmp_id 标识了了三个不同版本，其中dmp_id=1是对照组

ctr_data.drop(columns='dt',inplace=True)
ctr_data.head()

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2
3,1000006,0,1
4,1000006,0,3


In [3]:
# 找出对照组1的基准线,点击的百分比1.2%
ctr_data[ctr_data['dmp_id']==1]['label'].mean()

0.012492464908000114

In [ ]:
"""https://www.evanmiller.org/ab-testing/sample-size.html"""
# 如果想要实验组的效果提升1%，那么需要最小的样本量为2167

In [6]:
# 查看一下实验组的数据量
ctr_data['dmp_id'].value_counts()
# 大于最小样本量即可

1    1916035
2     412678
3     317245
Name: dmp_id, dtype: int64

In [11]:
# 查看一下三个组别的点击率情况：
print('对照组：',ctr_data[ctr_data.dmp_id==1].label.describe())
print('实验组2：',ctr_data[ctr_data.dmp_id==2].label.describe())
print('对照组3：',ctr_data[ctr_data.dmp_id==3].label.describe())

对照组： count    1.916035e+06
mean     1.249246e-02
std      1.110694e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: label, dtype: float64
实验组2： count    412678.000000
mean          0.015283
std           0.122677
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: label, dtype: float64
对照组3： count    317245.000000
mean          0.026131
std           0.159526
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: label, dtype: float64


In [ ]:
# 可以看到实验3的平均点击率为2.6131%，提升比较大；可以直接对实验3进行ABtest

In [17]:
import numpy as np
# 假设检验
# 1.零假设：实验组3的ctr >= 对照组1的ctr
#  备择假设： xxx
# 2.选择检验统计量：因为样本量较大，选择Z检验
# 3.计算
# 3.1两个组的点击率
p1 = ctr_data[ctr_data.dmp_id==1].label.mean()
p3 = ctr_data[ctr_data.dmp_id==3].label.mean()
# 3.2两个组的样本量
n1 = len(ctr_data[ctr_data.dmp_id==1])
n3 = len(ctr_data[ctr_data.dmp_id==3])
# 3.3计算点击率之和
pc = (p1*n1 + p3*n3)/(n1 + n3)
# 3.4计算检验统计量
z = (p1 - p3)/np.sqrt(pc*(1-pc)*(1/n1+1/n3))
print(z)

-59.66600946268368


In [20]:
# 设定显著性水平α为5%，对应的z分位数为：
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
z_alpha

-1.6448536269514729

In [ ]:
# 计算出的z统计量落在拒绝域内，拒绝原假设
# 因此，我们可以得出结论：在显著性水平为0.05时，可以拒绝原假设，实验组3的确实显著提升了点击率

In [ ]:
# 后续补充...